# Calculate Urban Metrics: Structure
using this example notebook: https://github.com/gboeing/osmnx-examples/blob/master/notebooks/06-stats-indicators-centrality.ipynb

OSMnx now caches server responses by default. This can be changed with ox.config(use_cache=False). The caching hash algorithm was changed for v1.0, invalidating old caches made with prior releases.

### Import libraries

In [ ]:
import logging

import geopandas as gpd
import pandas as pd
import osmnx as ox

In [ ]:
logging.basicConfig(filename="urban_structure_metrics.log", level=logging.INFO)

In [ ]:
import time

start_time = time.time()

In [ ]:
inputFolder = r"input_folder"
# define output folder
outputFolder = r"output_folder"

### You may need to use the 'Fix geometries' tool in QGIS beforehand to fix the geometries

In [ ]:
# read extents in WGS84
# shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\russia_urban_extents_merged_4326.shp"
# shpName = r"C:\repos\GOST_Urban\Notebooks\Implementations\eca_wo_rus_urban_clusters_ghs_pop_smooth_100k_4326_3.shp"
# shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UBZ_only_FUAs2_geom_fixed.shp"
# shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UZB_ghs_built_up_extents_4326\UZB_only_ghs_built_up_extents_4326_geom_fixed_greater_50k.shp"
# shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UZB_ghs_built_up_extents_4326\UZB_only_ghs_built_up_extents_4326_geom_fixed.shp"
# shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\ECA_wo_rus_urban_extents\eca_wo_rus_built_up_extents_4326.shp"
shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\all_urban_clusters_5k_up_4326.shp"

In [ ]:
input_shapes_gpd = gpd.read_file(shpName)

In [ ]:
input_shapes_gpd

In [ ]:
# input_shapes_gpd = input_shapes_gpd[137:]

In [ ]:
%%time

error_list = []

# for entry in input_shapes_gpd.head(2).iterrows():
for entry in input_shapes_gpd.iterrows():
    try:
        print(entry[0])
        # extract the geometry in GeoJSON format
        geometry = entry[1]["geometry"]  # list of shapely geometries

        gdf = gpd.GeoDataFrame()
        gdf["geometry"] = None
        gdf.loc[0, "geometry"] = geometry
        # Set the GeoDataFrame's coordinate system to WGS84
        gdf.crs = "epsg:4326"
        area = ox.project_gdf(gdf).unary_union.area
        print(area)

        G = ox.graph_from_polygon(geometry)
        # calculate basic and extended network stats, merge them together, and display
        stats = ox.basic_stats(G, area=area)
        print(stats)
        print(f"circuity_avg is: {stats['circuity_avg']}")

        # creates a temporary GDF for just the row's shape
        temp_gdf = input_shapes_gpd.loc[[entry[0]]]

        # print("print temp_gdf")
        # print(temp_gdf)

        # Put all metrics in a DataFrame
        metrics_scalar = {}
        metrics_scalar["circuity_avg"] = [stats["circuity_avg"]]
        metrics_scalar["intersection_density_km"] = [stats["intersection_density_km"]]
        metrics_scalar["street_density_km"] = [stats["street_density_km"]]
        metrics_df = pd.DataFrame(metrics_scalar)

        # print("print metrics_scalar")
        # print(metrics_scalar)

        # and concatinate it with the row's shape
        new_temp_gdf = pd.concat([temp_gdf.reset_index(drop=True), metrics_df], axis=1)

        # print("print new_temp_gdf")
        # print(new_temp_gdf)
        # print(entry[0])
        # put the results of each row into a new DataFrame
        if entry[0] == 0:
            print("new_temp_gdf")
            output_new_temp_gdf = new_temp_gdf
        else:
            output_new_temp_gdf = output_new_temp_gdf.append(
                new_temp_gdf, ignore_index=True
            )
    except:
        print(f"error with entry {entry[0]}")
        error_list.append(entry[0])
        # logging.INFO(f"error with entry {entry[0]}" + "\n" + f"entry information: {entry[1]}")

        # creates a temporary GDF for just the row's shape
        #         temp_gdf = input_shapes_gpd.loc[[entry[0]]]
        # Put all metrics in a DataFrame
        #         metrics_scalar = {}
        #         metrics_scalar['circuity_avg'] = 0
        #         metrics_scalar['intersection_density_km'] = 0
        #         metrics_scalar['street_density_km'] = 0
        #         metrics_df = pd.DataFrame(metrics_scalar)
        # and concatinate it with the row's shape
        #         new_temp_gdf = pd.concat([temp_gdf.reset_index(drop=True), metrics_df], axis=1)
        #         output_new_temp_gdf = output_new_temp_gdf.append(new_temp_gdf, ignore_index=True)
        continue

In [ ]:
error_list

In [ ]:
len(error_list)

In [ ]:
output_new_temp_gdf

In [ ]:
# output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\eca_metrics_results_russia"
# output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\eca_urban_metrics_results_wo_rus"
# output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UZB_only_GHS_FUAs_results"
# output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UZB_only_GHS_urban_extents_results"
# output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UZB_only_GHS_urban_extents_results_all"
# output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\eca_urban_metrics_results_wo_rus_all"
output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\all_urban_extents_results_5k_up"

In [ ]:
# save as CSV
# output_new_temp_gdf.to_csv(output + r"\ECA_russia_urban_metrics_100k_structure_to_146.csv")
# output_new_temp_gdf.to_csv(output + r"\UZB_only_urban_metrics_FUAs_structure.csv")
# output_new_temp_gdf.to_csv(output + r"\UZB_only_urban_metrics_urban_extents_structure.csv")
# output_new_temp_gdf.to_csv(output + r"\UZB_only_urban_metrics_urban_extents_all_structure.csv")
# output_new_temp_gdf.to_csv(output + r"\ECA_wo_rus_urban_metrics_urban_extents_all_structure.csv")
output_new_temp_gdf.to_csv(output + r"\all_urban_metrics_5k_up_structure.csv")

In [ ]:
print(f"total time to process: {time.time()-start_time}")